# LoC image analysis

This notebook is designed to take the raw output of a LoC data set, monolayer or bilayer, and:
 (trying to check if dask multichan works)

5. Localise and measure properties
6. Unite the localisations over the z-range
7. Save out the Z-tracks
8. Extract the maximum intensity from each cell

In [2]:
import os
import glob
from octopusheavy import DaskOctopusHeavyLoader
import napari
from skimage.io import imshow,  imsave, imread
import napari
import btrack
from tqdm.auto import tqdm
import dask.array as da

# Loading images

Define root path and individual experiment IDs

In [3]:
root_path = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy'

In [4]:
expt_IDs = ['co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/images/',
            'co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-VWF-iCAM1-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_70_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_150_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_262_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_130_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_253_/images/'
           ]

## Check to see what channels each expt has

In [5]:
for i, expt in enumerate(expt_IDs):
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    if 'MASK1' in [channel.name for channel in images.channels]:
        images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
        print(i, images.channels)

0 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
1 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
2 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
3 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
4 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
5 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]


In [6]:
images['CH1']

dask.array<stack, shape=(81, 2304, 2304), dtype=uint16, chunksize=(1, 2304, 2304), chunktype=numpy.ndarray>

### Defining properties to measure similarities of z-slices

In [8]:
props = ('axis_major_length', 
         'axis_minor_length', 
         'eccentricity', 
         'area', 
         #'intensity_image', 
         'orientation'
         'mean_intensity')

# this one works but for a very simple case of objects only bigger than 5000 area

In [10]:
for i, expt in tqdm(enumerate(expt_IDs), total = len(expt_IDs)):
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    multichannel_stack = da.stack([images['CH1'], images['CH2'], images['CH3'], images['CH4']], axis = -1)
    if 'MASK1' in [channel.name for channel in images.channels]: 
        ### create filename for tracks and objects 
        dirname = os.path.dirname(images.files('CH1')[0]).replace('images','')
        objects_fn = os.path.join(dirname, 'test_objects.hdf5')
        objects = btrack.utils.segmentation_to_objects(
            images['MASK1'], 
            multichannel_stack,
            properties = props,#('area', 'mean_intensity', 'intensity_image'), 
            use_weighted_centroid = False
        )
        ### prune objects 
        objects = [o for o in objects if o.properties['area']>50]
#         ### save out objects
#         with btrack.dataio.HDF5FileHandler(
#              objects_fn, 'w', obj_type='obj_type_1',
#         ) as hdf:
#             hdf.write_segmentation(images['MASK1'])
#             hdf.write_objects(objects)
        # initialise a tracker session using a context manager
        with btrack.BayesianTracker() as tracker:
            # configure the tracker using a config file
            tracker.configure('/home/dayn/analysis/BayesianTracker/models/particle_config.json')
            tracker.verbose = True
            ### set max search radius
            tracker.max_search_radius = 100
            # use visual features to track
            tracker.features = props
            # append the objects to be tracked
            tracker.append(objects)
            # set the volume (Z axis volume limits default to [-1e5, 1e5] for 2D data)
            tracker.volume=((0, 2304), (0, 2304), (-1e5, 1e5))
            # track them (in interactive mode)
#             tracker.track_interactive(step_size=100)
            tracker.track(tracking_updates =['visual', 'motion'], step_size=10)
            # generate hypotheses and run the global optimizer
            tracker.optimize()
            # get the tracks as a python list
            tracks = tracker.tracks
#             # filter tracks
#             tracks = [track for track in tracks if len(track) >= 3]
            # optional: get the data in a format for napari
            #   data, properties, graph = tracker.to_napari()
            tracker.export(f'test_tracks_{i}.hdf5', obj_type = 'obj_type_1')
            

  0%|          | 0/6 [00:00<?, ?it/s]

[INFO][2022/12/02 08:51:03 AM] Localizing objects from segmentation...
[INFO][2022/12/02 08:51:03 AM] Found intensity_image data
[INFO][2022/12/02 08:53:04 AM] Objects are of type: <class 'dict'>
[INFO][2022/12/02 08:53:05 AM] ...Found 70496 objects in 81 frames.
[INFO][2022/12/02 08:53:05 AM] Loaded btrack: /home/dayn/analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2022/12/02 08:53:05 AM] btrack (v0.5.0) library imported
[INFO][2022/12/02 08:53:05 AM] Starting BayesianTracker session
[INFO][2022/12/02 08:53:05 AM] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/particle_config.json
[INFO][2022/12/02 08:53:05 AM] Setting max_search_radius -> 100
[INFO][2022/12/02 08:53:05 AM] Setting features -> ('axis_major_length', 'axis_minor_length', 'eccentricity', 'area', 'orientation')
[INFO][2022/12/02 08:53:05 AM] Objects are of type: <class 'list'>
[INFO][2022/12/02 08:53:07 AM] Setting volume -> ((0, 2304), (0, 2304), (-100000.0, 100000.0))
[INFO][2022/12/02 

GLPK Integer Optimizer 5.0
45820 rows, 40546 columns, 58182 non-zeros
40546 integer variables, all of which are binary
Preprocessing...
22910 rows, 40546 columns, 58182 non-zeros
40546 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 22910
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
22910 rows, 40546 columns, 58182 non-zeros
*     0: obj =   1.453902221e+05 inf =   0.000e+00 (11701)
Perturbing LP to avoid stalling [976]...
*  5679: obj =   1.033994441e+05 inf =   0.000e+00 (4049) 1
Removing LP perturbation [10368]...
* 10368: obj =   8.977340703e+04 inf =   0.000e+00 (0) 6
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+ 10368: mip =     not found yet >=              -inf        (1; 0)
+ 10368: >>>>>   8.977340703e+04 >=   8.977340703e+04   0.0% (1; 0)
+ 10368: mip =  

[WARNING][2022/12/02 09:02:33 AM] Changing HDF filename to test_tracks_0.h5
[INFO][2022/12/02 09:02:33 AM] Opening HDF file: test_tracks_0.h5...
[INFO][2022/12/02 09:02:34 AM] Writing objects/obj_type_1
[INFO][2022/12/02 09:02:34 AM] Writing labels/obj_type_1
[INFO][2022/12/02 09:02:34 AM] Loading objects/obj_type_1 (59348, 5) (59348 filtered: None)
[INFO][2022/12/02 09:02:34 AM] Writing properties/obj_type_1/axis_major_length (59348,)
[INFO][2022/12/02 09:02:34 AM] Writing properties/obj_type_1/axis_minor_length (59348,)
[INFO][2022/12/02 09:02:34 AM] Writing properties/obj_type_1/eccentricity (59348,)
[INFO][2022/12/02 09:02:34 AM] Writing properties/obj_type_1/area (59348,)
[INFO][2022/12/02 09:02:34 AM] Writing properties/obj_type_1/orientation (59348,)
[INFO][2022/12/02 09:02:36 AM] Writing tracks/obj_type_1
[INFO][2022/12/02 09:02:36 AM] Writing dummies/obj_type_1
[INFO][2022/12/02 09:02:36 AM] Writing LBEP/obj_type_1
[INFO][2022/12/02 09:02:36 AM] Writing fates/obj_type_1
[INFO]

GLPK Integer Optimizer 5.0
41696 rows, 36620 columns, 52392 non-zeros
36620 integer variables, all of which are binary
Preprocessing...
20848 rows, 36620 columns, 52392 non-zeros
36620 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 20848
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
20848 rows, 36620 columns, 52392 non-zeros
*     0: obj =   1.365585702e+05 inf =   0.000e+00 (10624)
Perturbing LP to avoid stalling [1043]...
*  5328: obj =   9.601319491e+04 inf =   0.000e+00 (3793)
Removing LP perturbation [9605]...
*  9605: obj =   8.274791329e+04 inf =   0.000e+00 (0) 6
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  9605: mip =     not found yet >=              -inf        (1; 0)
+  9605: >>>>>   8.274791329e+04 >=   8.274791329e+04   0.0% (1; 0)
+  9605: mip =   8

...

[INFO][2022/12/02 09:54:31 AM] Completed optimization with 15723 tracks
[WARNING][2022/12/02 09:54:31 AM] Changing HDF filename to test_tracks_3.h5
[INFO][2022/12/02 09:54:31 AM] Opening HDF file: test_tracks_3.h5...
[INFO][2022/12/02 09:54:32 AM] Writing objects/obj_type_1
[INFO][2022/12/02 09:54:32 AM] Writing labels/obj_type_1
[INFO][2022/12/02 09:54:32 AM] Loading objects/obj_type_1 (60492, 5) (60492 filtered: None)
[INFO][2022/12/02 09:54:32 AM] Writing properties/obj_type_1/axis_major_length (60492,)
[INFO][2022/12/02 09:54:32 AM] Writing properties/obj_type_1/axis_minor_length (60492,)
[INFO][2022/12/02 09:54:32 AM] Writing properties/obj_type_1/eccentricity (60492,)
[INFO][2022/12/02 09:54:33 AM] Writing properties/obj_type_1/area (60492,)
[INFO][2022/12/02 09:54:33 AM] Writing properties/obj_type_1/orientation (60492,)
[INFO][2022/12/02 09:54:34 AM] Writing tracks/obj_type_1
[INFO][2022/12/02 09:54:34 AM] Writing dummies/obj_type_1
[INFO][2022/12/02 09:54:34 AM] Writing LBEP/o

GLPK Integer Optimizer 5.0
41708 rows, 34968 columns, 49082 non-zeros
34968 integer variables, all of which are binary
Preprocessing...
20854 rows, 34968 columns, 49082 non-zeros
34968 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 20854
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
20854 rows, 34968 columns, 49082 non-zeros
*     0: obj =   1.261914235e+05 inf =   0.000e+00 (7117)
Perturbing LP to avoid stalling [995]...
*  4613: obj =   9.377603168e+04 inf =   0.000e+00 (1527)
Removing LP perturbation [6252]...
*  6252: obj =   9.005883396e+04 inf =   0.000e+00 (0) 2
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  6252: mip =     not found yet >=              -inf        (1; 0)
+  6252: >>>>>   9.005883396e+04 >=   9.005883396e+04   0.0% (1; 0)
+  6252: mip =   9.0

[WARNING][2022/12/02 10:24:57 AM] Changing HDF filename to test_tracks_4.h5
[INFO][2022/12/02 10:24:57 AM] Opening HDF file: test_tracks_4.h5...
[INFO][2022/12/02 10:24:58 AM] Writing objects/obj_type_1
[INFO][2022/12/02 10:24:58 AM] Writing labels/obj_type_1
[INFO][2022/12/02 10:24:58 AM] Loading objects/obj_type_1 (82669, 5) (82669 filtered: None)
[INFO][2022/12/02 10:24:59 AM] Writing properties/obj_type_1/axis_major_length (82669,)
[INFO][2022/12/02 10:24:59 AM] Writing properties/obj_type_1/axis_minor_length (82669,)
[INFO][2022/12/02 10:24:59 AM] Writing properties/obj_type_1/eccentricity (82669,)
[INFO][2022/12/02 10:24:59 AM] Writing properties/obj_type_1/area (82669,)
[INFO][2022/12/02 10:24:59 AM] Writing properties/obj_type_1/orientation (82669,)
[INFO][2022/12/02 10:25:00 AM] Writing tracks/obj_type_1
[INFO][2022/12/02 10:25:00 AM] Writing dummies/obj_type_1
[INFO][2022/12/02 10:25:00 AM] Writing LBEP/obj_type_1
[INFO][2022/12/02 10:25:00 AM] Writing fates/obj_type_1
[INFO]

GLPK Integer Optimizer 5.0
35476 rows, 29307 columns, 40876 non-zeros
29307 integer variables, all of which are binary
Preprocessing...
17738 rows, 29307 columns, 40876 non-zeros
29307 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 17738
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
17738 rows, 29307 columns, 40876 non-zeros
*     0: obj =   1.003691239e+05 inf =   0.000e+00 (5420)
Perturbing LP to avoid stalling [1430]...
Removing LP perturbation [4789]...
*  4789: obj =   7.246010750e+04 inf =   0.000e+00 (0) 1
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  4789: mip =     not found yet >=              -inf        (1; 0)
+  4789: >>>>>   7.246010750e+04 >=   7.246010750e+04   0.0% (1; 0)
+  4789: mip =   7.246010750e+04 >=     tree is empty   0.0% (0; 1)
INTEGER O

[WARNING][2022/12/02 10:46:40 AM] Changing HDF filename to test_tracks_5.h5
[INFO][2022/12/02 10:46:40 AM] Opening HDF file: test_tracks_5.h5...
[INFO][2022/12/02 10:46:41 AM] Writing objects/obj_type_1
[INFO][2022/12/02 10:46:41 AM] Writing labels/obj_type_1
[INFO][2022/12/02 10:46:41 AM] Loading objects/obj_type_1 (58357, 5) (58357 filtered: None)
[INFO][2022/12/02 10:46:41 AM] Writing properties/obj_type_1/axis_major_length (58357,)
[INFO][2022/12/02 10:46:41 AM] Writing properties/obj_type_1/axis_minor_length (58357,)
[INFO][2022/12/02 10:46:41 AM] Writing properties/obj_type_1/eccentricity (58357,)
[INFO][2022/12/02 10:46:41 AM] Writing properties/obj_type_1/area (58357,)
[INFO][2022/12/02 10:46:41 AM] Writing properties/obj_type_1/orientation (58357,)
[INFO][2022/12/02 10:46:42 AM] Writing tracks/obj_type_1
[INFO][2022/12/02 10:46:42 AM] Writing dummies/obj_type_1
[INFO][2022/12/02 10:46:42 AM] Writing LBEP/obj_type_1
[INFO][2022/12/02 10:46:42 AM] Writing fates/obj_type_1
[INFO]

In [1]:
with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure('/home/dayn/analysis/BayesianTracker/models/particle_config.json')
        tracker.verbose = True
        ### set max search radius
        tracker.max_search_radius = 100
        # use visual features to track
        tracker.features = props
        # append the objects to be tracked
        tracker.append(objects)
        # set the volume (Z axis volume limits default to [-1e5, 1e5] for 2D data)
        tracker.volume=((0, 2304), (0, 2304), (-1e5, 1e5))
        # track them (in interactive mode)
        #             tracker.track_interactive(step_size=100)
        tracker.track(tracking_updates =['visual', 'motion'], step_size=10)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # get the tracks as a python list
        tracks = tracker.tracks
        #             # filter tracks
        #             tracks = [track for track in tracks if len(track) >= 3]
        # optional: get the data in a format for napari
        #   data, properties, graph = tracker.to_napari()
        tracker.export('test.h5', obj_type='obj_type_1')

NameError: name 'btrack' is not defined

In [13]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:
    # configure the tracker using a config file
    tracker.configure('/home/dayn/analysis/BayesianTracker/models/particle_config.json')
    ### filter the objects
    objects = [o for o in objects if o.properties['area']>50.]
    # append the objects to be tracked
    tracker.append(objects)
    # set the volume (Z axis volume limits default to [-1e5, 1e5] for 2D data)
    tracker.volume = ((0, 2304), (0, 2304))
    # track them (in interactive mode)
    tracker.track_interactive(step_size=10)
    # generate hypotheses and run the global optimizer
    tracker.optimize()
    # get the tracks as a python list
    tracks = tracker.tracks
    # filter tracks
    tracks = [track for track in tracks if len(track) >= 3]
    # optional: get the data in a format for napari
    #   data, properties, graph = tracker.to_napari()
    tracker.export(objects_fn.replace('objects', 'tracks'), obj_type="obj_type_1")

[INFO][2022/12/02 07:36:14 AM] Loaded btrack: /home/dayn/analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2022/12/02 07:36:14 AM] btrack (v0.5.0) library imported
[INFO][2022/12/02 07:36:14 AM] Starting BayesianTracker session
[INFO][2022/12/02 07:36:14 AM] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/particle_config.json
[INFO][2022/12/02 07:36:14 AM] Objects are of type: <class 'list'>
[WARNING][2022/12/02 07:36:14 AM] `track_interactive` will be deprecated. Use `track` instead.
[INFO][2022/12/02 07:36:14 AM] Starting tracking... 
[INFO][2022/12/02 07:36:14 AM] Update using: ['MOTION']
[INFO][2022/12/02 07:36:14 AM] Tracking objects in frames 0 to 9 (of 81)...
[INFO][2022/12/02 07:36:17 AM]  - Timing (Bayesian updates: 967.82ms, Linking: 10.29ms)
[INFO][2022/12/02 07:36:17 AM]  - Probabilities (Link: 0.81557, Lost: 0.99984)
[INFO][2022/12/02 07:36:17 AM]  - Stats (Active: 1222, Lost: 1255, Conflicts resolved: 1021)
[INFO][2022/12/02 07:36:17 AM] Tra

GLPK Integer Optimizer 5.0
51220 rows, 46701 columns, 67792 non-zeros
46701 integer variables, all of which are binary
Preprocessing...
25610 rows, 46701 columns, 67792 non-zeros
46701 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 25610
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
25610 rows, 46701 columns, 67792 non-zeros
*     0: obj =   1.629079900e+05 inf =   0.000e+00 (15315)
Perturbing LP to avoid stalling [1085]...
*  4793: obj =   1.219892482e+05 inf =   0.000e+00 (8741)
*  8749: obj =   1.067840345e+05 inf =   0.000e+00 (3616) 3
* 13302: obj =   9.377980305e+04 inf =   0.000e+00 (16) 7
Removing LP perturbation [13318]...
* 13318: obj =   9.377977700e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+ 13318: mip =     not found yet >=  

[INFO][2022/12/02 07:41:50 AM] Completed optimization with 10921 tracks
[INFO][2022/12/02 07:41:50 AM] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/tracks.h5...
[INFO][2022/12/02 07:41:50 AM] Writing objects/obj_type_1
[INFO][2022/12/02 07:41:50 AM] Writing labels/obj_type_1
[INFO][2022/12/02 07:41:50 AM] Loading objects/obj_type_1 (59348, 5) (59348 filtered: None)
[INFO][2022/12/02 07:41:51 AM] Writing properties/obj_type_1/area (59348,)
[INFO][2022/12/02 07:41:51 AM] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel 

OSError: [Errno 95] Unable to extend file properly, errno = 95, error message = 'operation not supported' (file write failed: time = Fri Dec  2 07:41:51 2022
, filename = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/tracks.h5', file descriptor = 59, errno = 95, error message = 'Operation not supported', buf = 0x5617dba54bb8, total write size = 3368, bytes this sub-write = 3368, bytes actually written = 18446744073709551615, offset = 0)

In [10]:
for i, expt in tqdm(enumerate(expt_IDs), total = len(expt_IDs)):
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    multichannel_stack = da.stack([images['CH1'], images['CH2'], images['CH3'], images['CH4']], axis = -1)
    if 'MASK1' in [channel.name for channel in images.channels]: 
        ### create filename for tracks and objects 
        dirname = os.path.dirname(images.files('CH1')[0]).replace('images','')
        objects_fn = os.path.join(dirname, 'objects.h5')
        objects = btrack.utils.segmentation_to_objects(
            images['MASK1'], 
            multichannel_stack,
            properties = props,#('area', 'mean_intensity', 'intensity_image'), 
            use_weighted_centroid = False
        )
#         ### save out objects
#         with btrack.dataio.HDF5FileHandler(
#         #      f'{root_dir}/{expt}/{pos}/objects.h5', 'w', obj_type='obj_type_1',
#              objects_fn, 'w', obj_type='obj_type_1',
#         ) as hdf:
#             hdf.write_segmentation(images['MASK1'])
#             hdf.write_objects(objects)
    ### redefine features as the mean intensity no longer exists
    props = (
            'axis_major_length', 
             'axis_minor_length', 
             'eccentricity', 
             'area', 
             #'intensity_image', 
             'orientation'
    )


    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:

        # configure the tracker using a config file
        tracker.configure_from_file(
            '/home/dayn/analysis/BayesianTracker/models/particle_config.json'
        )
        tracker.verbose = True

        tracker.max_search_radius = 100

        # use visual features to track
        tracker.features = props

        # append the objects to be tracked
        tracker.append(objects)

        # set the volume
        tracker.volume=((0, 2304), (0, 2304), (-1e5, 1e5))

        # track them (in interactive mode)
#         tracker.track_interactive(step_size=10)
        tracker.track(tracking_updates =['visual', 'motion'], step_size=10)

        # generate hypotheses and run the global optimizer
        tracker.optimize()
        
        ### export filename
        tracks_fn = objects_fn.replace('objects', 'tracks')

        tracker.export(tracks_fn, obj_type='obj_type_1')

#     #     # get the tracks in a format for napari visualization (optional)
#         visualise_tracks, properties, graph = tracker.to_napari(ndim=2)

        tracks = tracker.tracks

  0%|          | 0/6 [00:00<?, ?it/s]

[INFO][2022/12/02 07:59:56 AM] Localizing objects from segmentation...
[INFO][2022/12/02 07:59:56 AM] Found intensity_image data
[INFO][2022/12/02 08:02:06 AM] Objects are of type: <class 'dict'>
[INFO][2022/12/02 08:02:07 AM] ...Found 70496 objects in 81 frames.
[INFO][2022/12/02 08:02:07 AM] Loaded btrack: /home/dayn/analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2022/12/02 08:02:07 AM] btrack (v0.5.0) library imported
[INFO][2022/12/02 08:02:07 AM] Starting BayesianTracker session
[INFO][2022/12/02 08:02:07 AM] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/particle_config.json
[INFO][2022/12/02 08:02:07 AM] Setting max_search_radius -> 100
[INFO][2022/12/02 08:02:07 AM] Setting features -> ('axis_major_length', 'axis_minor_length', 'eccentricity', 'area', 'orientation')
[INFO][2022/12/02 08:02:07 AM] Objects are of type: <class 'list'>
[INFO][2022/12/02 08:02:09 AM] Setting volume -> ((0, 2304), (0, 2304), (-100000.0, 100000.0))
[INFO][2022/12/02 

GLPK Integer Optimizer 5.0
73188 rows, 77851 columns, 119108 non-zeros
77851 integer variables, all of which are binary
Preprocessing...
36594 rows, 77851 columns, 119108 non-zeros
77851 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 36594
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
36594 rows, 77851 columns, 119108 non-zeros
*     0: obj =   2.318488706e+05 inf =   0.000e+00 (34223)
Perturbing LP to avoid stalling [999]...
*  3236: obj =   2.068889956e+05 inf =   0.000e+00 (25831)
*  6330: obj =   1.853418583e+05 inf =   0.000e+00 (20177) 1
*  9716: obj =   1.570767996e+05 inf =   0.000e+00 (16305)
* 12453: obj =   1.413846207e+05 inf =   0.000e+00 (12926) 4
* 14675: obj =   1.364401930e+05 inf =   0.000e+00 (9470) 5
* 16906: obj =   1.318089870e+05 inf =   0.000e+00 (6618) 4
* 19368: obj =   1.240419435e+05 in

[INFO][2022/12/02 08:20:22 AM] Completed optimization with 15615 tracks
[INFO][2022/12/02 08:20:22 AM] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/tracks.h5...
[INFO][2022/12/02 08:20:22 AM] Ending BayesianTracker session


OSError: [Errno 95] Unable to open file (unable to open file: name = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/tracks.h5', errno = 95, error message = 'Operation not supported', flags = 1, o_flags = 2)

# test tracks to save out

In [26]:
test_objects = [obj for obj in objects if obj.properties['area']>4000]

In [27]:
len(test_objects)

802

In [ ]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file(
        '/home/dayn/analysis/BayesianTracker/models/particle_config.json'
    )
    tracker.verbose = True

    tracker.max_search_radius = 100

    # use visual features to track
    tracker.features = props

    # append the objects to be tracked
    tracker.append(objects)

    # set the volume
    tracker.volume=((0, 2304), (0, 2304), (-1e5, 1e5))

    # track them (in interactive mode)
#         tracker.track_interactive(step_size=10)
    tracker.track(tracking_updates =['visual', 'motion'], step_size=10)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    ### export filename
    tracks_fn = objects_fn.replace('objects', 'tracks')

    tracker.export('test_tracks.hdf5', obj_type='obj_type_1')

#     #     # get the tracks in a format for napari visualization (optional)
#         visualise_tracks, properties, graph = tracker.to_napari(ndim=2)

    tracks = tracker.tracks

[INFO][2022/12/02 08:34:26 AM] Loaded btrack: /home/dayn/analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2022/12/02 08:34:26 AM] btrack (v0.5.0) library imported
[INFO][2022/12/02 08:34:26 AM] Starting BayesianTracker session
[INFO][2022/12/02 08:34:26 AM] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/particle_config.json
[INFO][2022/12/02 08:34:26 AM] Setting max_search_radius -> 100
[INFO][2022/12/02 08:34:26 AM] Setting features -> ('axis_major_length', 'axis_minor_length', 'eccentricity', 'area', 'orientation')
[INFO][2022/12/02 08:34:26 AM] Objects are of type: <class 'list'>
[INFO][2022/12/02 08:34:27 AM] Setting volume -> ((0, 2304), (0, 2304), (-100000.0, 100000.0))
[INFO][2022/12/02 08:34:27 AM] Starting tracking... 
[INFO][2022/12/02 08:34:27 AM] Update using: ['VISUAL', 'MOTION']
[INFO][2022/12/02 08:34:27 AM] Tracking objects in frames 0 to 9 (of 81)...
[INFO][2022/12/02 08:34:33 AM]  - Timing (Bayesian updates: 810.44ms, Linking: 9.88ms)


In [30]:
tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,axis_major_length,axis_minor_length,orientation,eccentricity,area
0,2,4,1696.218139,1428.613919,0.000000,2,2,5,0,False,145.007709,53.097867,1.116294,0.930547,5877.0
1,2,5,1707.525141,1433.861075,0.000000,2,2,5,0,False,109.475853,55.086582,1.112422,0.864179,4614.0
2,2,6,1717.238732,1439.029782,1.233340,2,2,5,0,True,NaN,NaN,NaN,NaN,NaN
3,2,7,1726.952324,1444.198489,2.466679,2,2,5,0,True,NaN,NaN,NaN,NaN,NaN
4,2,8,1736.665915,1449.367196,3.700019,2,2,5,0,True,NaN,NaN,NaN,NaN,NaN
5,2,9,1746.379507,1454.535903,4.933359,2,2,5,0,True,NaN,NaN,NaN,NaN,NaN
6,2,10,1697.792267,1429.614938,0.000000,2,2,5,0,False,147.013327,51.911339,1.139960,0.935583,5690.0
7,2,11,1698.435411,1427.370334,0.000000,2,2,5,0,False,145.287340,55.755082,1.124484,0.923434,6162.0
8,2,12,1696.186360,1427.305948,0.000000,2,2,5,0,False,142.611947,57.787924,1.129322,0.914223,6305.0


In [ ]:
test

In [17]:
tracks_fn = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/tracks.h5' 

In [16]:
with btrack.dataio.HDF5FileHandler(
     'test_tracks.h5', 'w', obj_type='obj_type_1',
) as hdf:
#     hdf.write_segmentation(masks)
    hdf.write_objects(objects)
#     hdf.write_tracks(tracks)

[INFO][2022/12/02 08:23:33 AM] Opening HDF file: test_tracks.h5...
[INFO][2022/12/02 08:23:34 AM] Writing objects/obj_type_1
[INFO][2022/12/02 08:23:34 AM] Writing labels/obj_type_1
[INFO][2022/12/02 08:23:34 AM] Loading objects/obj_type_1 (70496, 5) (70496 filtered: None)
[INFO][2022/12/02 08:23:35 AM] Writing properties/obj_type_1/axis_major_length (70496,)
[INFO][2022/12/02 08:23:35 AM] Writing properties/obj_type_1/axis_minor_length (70496,)
[INFO][2022/12/02 08:23:35 AM] Writing properties/obj_type_1/eccentricity (70496,)
[INFO][2022/12/02 08:23:35 AM] Writing properties/obj_type_1/area (70496,)
[INFO][2022/12/02 08:23:35 AM] Writing properties/obj_type_1/orientation (70496,)
[INFO][2022/12/02 08:23:35 AM] Closing HDF file: test_tracks.h5


In [19]:
objects[0]

,ID,x,y,z,t,dummy,states,label,axis_major_length,axis_minor_length,eccentricity,area,orientation
0,0,8.078947,5.473684,0.0,0,False,7,5,11.460551,6.562872,0.819801,38,-0.301347


In [18]:
with btrack.dataio.HDF5FileHandler(tracks_fn, "r", obj_type="obj_type_1") as hdf:
    new_tracks = hdf.tracks
#     new_data, new_properties, new_graph = btrack.utils.tracks_to_napari(new_tracks, ndim=2)

[INFO][2022/12/02 08:25:44 AM] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/tracks.h5...
[ERROR][2022/12/02 08:25:44 AM] Tracks not found in /run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/tracks.h5
[INFO][2022/12/02 08:25:44 AM] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_202208

In [15]:
tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,mean_intensity-1,mean_intensity-3,mean_intensity-0,area,mean_intensity-2
0,84,0,1805.295000,1010.710000,0.000000,84,84,5,0,False,117.715000,135.440000,113.695000,200.0,125.475000
1,84,1,1805.295000,1010.710000,0.000000,84,84,5,0,True,NaN,NaN,NaN,NaN,NaN
2,84,2,1806.155039,1006.994832,0.000000,84,84,5,0,False,120.648579,139.351421,113.777778,387.0,133.271318
3,84,3,1809.155172,1007.473180,0.000000,84,84,5,0,False,123.747126,146.607280,116.134100,522.0,144.057471
4,84,4,1809.585534,1000.601607,0.000000,84,84,5,0,False,128.887486,149.845006,121.262916,871.0,153.838117
5,84,5,1810.428263,998.982740,0.000000,84,84,5,0,False,136.749730,158.226537,124.181230,927.0,165.513484
6,84,6,1811.001491,997.470002,0.012450,84,84,5,0,True,NaN,NaN,NaN,NaN,NaN
7,84,7,1811.729433,996.912249,0.000000,84,84,5,0,False,164.168190,179.506399,153.382084,547.0,218.354662
8,84,8,1812.311369,995.558270,-0.012520,84,84,5,0,True,NaN,NaN,NaN,NaN,NaN
9,84,9,1812.893306,994.204290,-0.025039,84,84,5,0,True,NaN,NaN,NaN,NaN,NaN


## Checking tracks prior to plotting single cell info

In [ ]:
import napari

In [ ]:
viewer = napari.Viewer()

viewer.add_image(ch0, colormap='blue', blending = 'additive')
viewer.add_image(ch3, colormap= 'gray', blending = 'additive')
viewer.add_image(ch1, colormap='red', blending = 'additive')
viewer.add_image(ch2, colormap= 'green', blending = 'additive')

viewer.add_labels(mask_stack)

viewer.add_tracks(ch1_data)
viewer.add_tracks(ch2_data)
